In [1]:
from new_SiameseNet import *

Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is disabled, CuDNN not available)
/home/ubuntu/anaconda/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [12]:
# Prepare data
print 'Getting CIFAR10 data...'

data = get_CIFAR10_data()
val_set = [0, 1, 8, 9] # hold out the vehicles

old_x_train, old_y_train = data['X_train'], data['y_train']
old_x_val,   old_y_val   = data['X_val'],   data['y_val']

print old_x_train.shape
print old_x_val.shape

N = old_x_train.shape[0]

# in zero-shot, we want to hold out some of the classes from training

x_train = []
y_train = []
x_val = []
y_val = []

for i in range(N):
    x = np.expand_dims(old_x_train[i],0)
    y = old_y_train[i]
    if old_y_train[i] in val_set:
        x_val.append(x)
        y_val.append(y)
    else:
        x_train.append(x)
        y_train.append(y)
        
for i in range(old_x_val.shape[0]):
    x = np.expand_dims(old_x_val[i],0)
    y = old_y_val[i]
    if old_y_val[i] in val_set:
        x_val.append(x)
        y_val.append(y)
    else:
        x_train.append(x)
        y_train.append(y)
        
x_train = np.vstack(x_train)
#y_train = np.vstack(y_train)
x_val = np.vstack(x_val)
#y_val = np.vstack(y_val)
print x_train.shape
print x_val.shape

Getting CIFAR10 data...
(49000, 3, 32, 32)
(1000, 3, 32, 32)
(30000, 3, 32, 32)
(20000, 3, 32, 32)


In [13]:
# Specify structure of Siamese part of SiameseNet
# This part needs to be improved. I'm kind of just using random layers.
init = 'glorot_uniform'
in_shp = (3,32,32)
seq = Sequential()
seq.add(BatchNormalization(epsilon=1e-7,
                            mode=0,
                            axis=1,
                            momentum=0.9,
                            weights=None,
                            input_shape=in_shp))
for _ in range(3):
    seq.add(Convolution2D(10, 3, 3, init=init, border_mode='same'))
    seq.add(Activation('relu'))
seq.add(Flatten())
seq.add(Dense(128))
layers = seq

sn = SiameseNet(layers, input_shape=(3, 32, 32), verbose=False)
sn.compile()

In [14]:
def _train_sn(sn, x_train, y_train, x_val, y_val, filepath):
    d_val = invert_dataset(x_val,  y_val)
    d_train = invert_dataset(x_train,  y_train)
    num_ep = 2
    history = sn.fit(*generate_data(x_train, d_train), 
            validation_data=generate_data(x_val, d_val),
            nb_epoch=num_ep)
    return history

history = _train_sn(sn, x_train, y_train, x_val, y_val, filepath='weights.h5')

IndexError: too many indices for array